# Proximal Policy Optimization

**Author:** [Ilias Chrysovergis](https://twitter.com/iliachry)<br>
**Date created:** 2021/06/24<br>
**Last modified:** 2021/06/24<br>
**Description:** Implementation of a Proximal Policy Optimization agent for the CartPole-v0 environment.

## Introduction

This code example solves the CartPole-v0 environment using a Proximal Policy Optimization (PPO) agent.

### CartPole-v0

A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track.
The system is controlled by applying a force of +1 or -1 to the cart.
The pendulum starts upright, and the goal is to prevent it from falling over.
A reward of +1 is provided for every timestep that the pole remains upright.
The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.
After 200 steps the episode ends. Thus, the highest return we can get is equal to 200.

[CartPole-v0](https://gym.openai.com/envs/CartPole-v0/)

### Proximal Policy Optimization

PPO is a policy gradient method and can be used for environments with either discrete or continuous action spaces.
It trains a stochastic policy in an on-policy way. Also, it utilizes the actor critic method. The actor maps the
observation to an action and the critic gives an expectation of the rewards of the agent for the observation given.
Firstly, it collects a set of trajectories for each epoch by sampling from the latest version of the stochastic policy.
Then, the rewards-to-go and the advantage estimates are computed in order to update the policy and fit the value function.
The policy is updated via a stochastic gradient ascent optimizer, while the value function is fitted via some gradient descent algorithm.
This procedure is applied for many epochs until the environment is solved.

![Algorithm](https://i.imgur.com/rd5tda1.png)

- [PPO Original Paper](https://arxiv.org/pdf/1707.06347.pdf)
- [OpenAI Spinning Up docs - PPO](https://spinningup.openai.com/en/latest/algorithms/ppo.html)

### Note

This code example uses Keras and Tensorflow v2. It is based on the PPO Original Paper,
the OpenAI's Spinning Up docs for PPO, and the OpenAI's Spinning Up implementation of PPO using Tensorflow v1.

[OpenAI Spinning Up Github - PPO](https://github.com/openai/spinningup/blob/master/spinup/algos/tf1/ppo/ppo.py)

## Libraries

For this example the following libraries are used:

1. `numpy` for n-dimensional arrays
2. `tensorflow` and `keras` for building the deep RL PPO agent
3. `gym` for getting everything we need about the environment
4. `scipy.signal` for calculating the discounted cumulative sums of vectors

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gym
import scipy.signal
import time
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

## Functions and class

In [ ]:

def discounted_cumulative_sums(x, discount):
    # Discounted cumulative sums of vectors for computing rewards-to-go and advantage estimates
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]


class Buffer:
    # Buffer for storing trajectories
    def __init__(self, observation_dimensions, size, gamma=0.99, lam=0.95):
        # Buffer initialization
        self.observation_buffer = np.zeros(
            (size, observation_dimensions), dtype=np.float32
        )
        self.action_buffer = np.zeros(size, dtype=np.int32)
        self.advantage_buffer = np.zeros(size, dtype=np.float32)
        self.reward_buffer = np.zeros(size, dtype=np.float32)
        self.return_buffer = np.zeros(size, dtype=np.float32)
        self.value_buffer = np.zeros(size, dtype=np.float32)
        self.logprobability_buffer = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.pointer, self.trajectory_start_index = 0, 0

    def store(self, observation, action, reward, value, logprobability):
        # Append one step of agent-environment interaction
        self.observation_buffer[self.pointer] = observation
        self.action_buffer[self.pointer] = action
        self.reward_buffer[self.pointer] = reward
        self.value_buffer[self.pointer] = value
        self.logprobability_buffer[self.pointer] = logprobability
        self.pointer += 1

    def finish_trajectory(self, last_value=0):
        # Finish the trajectory by computing advantage estimates and rewards-to-go
        path_slice = slice(self.trajectory_start_index, self.pointer)
        rewards = np.append(self.reward_buffer[path_slice], last_value)
        values = np.append(self.value_buffer[path_slice], last_value)

        deltas = rewards[:-1] + self.gamma * values[1:] - values[:-1]

        self.advantage_buffer[path_slice] = discounted_cumulative_sums(
            deltas, self.gamma * self.lam
        )
        self.return_buffer[path_slice] = discounted_cumulative_sums(
            rewards, self.gamma
        )[:-1]

        self.trajectory_start_index = self.pointer

    def get(self):
        # Get all data of the buffer and normalize the advantages
        self.pointer, self.trajectory_start_index = 0, 0
        advantage_mean, advantage_std = (
            np.mean(self.advantage_buffer),
            np.std(self.advantage_buffer),
        )
        self.advantage_buffer = (self.advantage_buffer - advantage_mean) / advantage_std
        return (
            self.observation_buffer,
            self.action_buffer,
            self.advantage_buffer,
            self.return_buffer,
            self.logprobability_buffer,
        )


def mlp(x, sizes, activation=tf.nn.relu, output_activation=None):
    # Build a feedforward neural network
    for size in sizes[:-1]:
        x = layers.Dense(units=size, activation=activation)(x)
    return layers.Dense(units=sizes[-1], activation=output_activation)(x)


def logprobabilities(logits, a):
    # Compute the log-probabilities of taking actions a by using the logits (i.e. the output of the actor)
    logprobabilities_all = tf.nn.log_softmax(logits)
    logprobability = tf.reduce_sum(
        tf.one_hot(a, num_actions) * logprobabilities_all, axis=1
    )
    return logprobability


# Sample action from actor
@tf.function
def sample_action(observation):
    logits = actor(observation)
    action = tf.squeeze(tf.random.categorical(logits, 1), axis=1)
    return logits, action


# Train the policy by maxizing the PPO-Clip objective
@tf.function
def train_policy(
    observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
):

    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        ratio = tf.exp(
            logprobabilities(actor(observation_buffer), action_buffer)
            - logprobability_buffer
        )
        min_advantage = tf.where(
            advantage_buffer > 0,
            (1 + clip_ratio) * advantage_buffer,
            (1 - clip_ratio) * advantage_buffer,
        )

        policy_loss = -tf.reduce_mean(
            tf.minimum(ratio * advantage_buffer, min_advantage)
        )
    policy_grads = tape.gradient(policy_loss, actor.trainable_variables)
    policy_optimizer.apply_gradients(zip(policy_grads, actor.trainable_variables))

    kl = tf.reduce_mean(
        logprobability_buffer
        - logprobabilities(actor(observation_buffer), action_buffer)
    )
    kl = tf.reduce_sum(kl)
    return kl


# Train the value function by regression on mean-squared error
@tf.function
def train_value_function(observation_buffer, return_buffer):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        value_loss = tf.reduce_mean((return_buffer - critic(observation_buffer)) ** 2)
    value_grads = tape.gradient(value_loss, critic.trainable_variables)
    value_optimizer.apply_gradients(zip(value_grads, critic.trainable_variables))


## Hyperparameters

In [ ]:
# Hyperparameters of the PPO algorithm
steps_per_epoch = 500
epochs = 1000
gamma = 0.99
clip_ratio = 0.2
policy_learning_rate = 3e-4
value_function_learning_rate = 1e-3
train_policy_iterations = 80
train_value_iterations = 80
lam = 0.95
target_kl = 0.01
hidden_sizes = (400, 300)

# True if you want to render the environment
render = False

In [ ]:
def plot_euler_angles(t, roll, pitch, yaw, filename=None):
  fig1 = plt.figure(figsize=(8,8))
  ax1 = fig1.add_subplot(1,1,1)
  ax1.set_title("Euler Angles")
  ax1.plot(t, roll, label = 'roll', color = 'red')
  ax1.plot(t, pitch, label = 'pitch', color = 'green')
  ax1.plot(t, yaw, label = 'yaw', color = 'blue')
  #ax1.set_ybound(-10, 10)
  ax1.set_ylabel(r'angles, [deg]')
  ax1.set_xlabel(r't, [s]')
  ax1.grid(True)
  ax1.legend()
  fig1.show()
  if filename is not None:
    fig1.savefig(f'{filename}')

def plot_euler_angles_radial(t, roll, pitch, yaw, filename=None):
  fig1 = plt.figure(figsize=(8,8))
  ax1 = fig1.add_subplot(1,1,1, polar=True)
  ax1.set_title("Euler Angles")
  ax1.plot(roll, t, label = 'roll', color = 'red')
  ax1.plot(pitch, t, label = 'pitch', color = 'green')
  ax1.plot(yaw, t, label = 'yaw', color = 'blue')
  #ax1.set_ylabel(r'angles, [deg]')
  #ax1.set_xlabel(r't, [s]')
  ax1.grid(True)
  ax1.legend()
  fig1.show()
  if filename is not None:
    fig1.savefig(f'{filename}')

def plot_phase_diagram(roll, pitch, yaw, omega_, filename=None):
  fig2 = plt.figure(figsize=(8,8))
  ax2 = fig2.add_subplot(1,1,1)
  ax2.set_title("Phase diagram")
  ax2.plot(roll, omega_[0], label = 'roll', color = 'red')
  ax2.plot(pitch, omega_[1], label = 'pitch', color = 'green')
  ax2.plot(yaw, omega_[2], label = 'yaw', color = 'blue')
  ax2.plot([0],[0], 'o', markersize=8 , label='Destination', color = 'magenta')
  ax2.set_xlabel(r'$\alpha$')
  ax2.set_ylabel(r'$\omega$')
  ax2.legend()
  ax2.grid()
  fig2.show()
  if filename is not None:
    fig2.savefig(f'{filename}')

In [ ]:
# Вспомогательные функции

def normalize(obj):

    return obj / np.linalg.norm(obj)


def cross_product(a, b):

    def check_dimensions(vec, string):

        if vec.ndim != 1:
            raise Exception("The {} input is not a vector".format(string))
        if len(vec) != 3:
            raise Exception("Wrong number of coordinates in the {0} vector: {1}, should be 3".format(string, len(vec)))

    check_dimensions(a, 'first')
    check_dimensions(b, 'second')

    return np.array([a[1]*b[2]-a[2]*b[1], a[2]*b[0]-a[0]*b[2], a[0]*b[1] - a[1]*b[0]])

def quat_product(q1, q2):

    def check_dimensions(q, string):

        if q.ndim != 1:
            raise Exception("The {} input is not a quaternion".format(string))
        if len(q) != 4:
            raise Exception("Wrong number of coordinates in the {0} quaternion: {1}, should be 4".format(string, len(q)))

    check_dimensions(q1, 'first')
    check_dimensions(q2, 'second')

    q = np.zeros(4)
    q[0] = q1[0] * q2[0] - q1[1:].dot(q2[1:])
    q[1:] = q1[0] * q2[1:] + q2[0] * q1[1:] + cross_product(q1[1:], q2[1:])

    return q

def rotate_vec_with_quat(q, vec):

    def check_dimensions(obj, is_quat):

        if obj.ndim != 1:
            raise Exception("Not a {}".format('quaternion' * is_quat + 'vector' * (1 - is_quat)))
        if len(obj) != (3 + 1 * is_quat):
            raise Exception("Wrong number of coordinates in the {0}: {1}, should be {2}"
                            .format('quaternion' * is_quat + 'vector' * (1 - is_quat), len(obj), 3 + 1 * is_quat))

    check_dimensions(q, True)
    check_dimensions(vec, False)

    q = quat_conjugate(q)

    qxvec = cross_product(q[1:], vec)

    return q[1:].dot(vec) * q[1:] + q[0]**2. * vec + 2. * q[0] * qxvec + cross_product(q[1:], qxvec)

def quat2rpy(q0, q1, q2, q3):

    roll = np.arctan2(2. * (q0 * q1 + q2 * q3), 1. - 2. * (q1**2 + q2**2))
    pitch = np.arcsin(2. * (q0 * q2 - q1 * q3))
    yaw = np.arctan2(2. * (q0 * q3 + q1 * q2), 1. - 2. * (q2**2 + q3**2))

    return [roll, pitch, yaw]

def quat2rpy_deg(q0, q1, q2, q3):

    roll = np.arctan2(2. * (q0 * q1 + q2 * q3), 1. - 2. * (q1**2 + q2**2))*180/np.pi
    pitch = np.arcsin(2. * (q0 * q2 - q1 * q3))*180/np.pi
    yaw = np.arctan2(2. * (q0 * q3 + q1 * q2), 1. - 2. * (q2**2 + q3**2))*180/np.pi

    return [roll, pitch, yaw]

def quat_conjugate(q):

    q_new = np.copy(q)
    q_new[1:] = q_new[1:] * -1.

    return q_new

def gravity_gradient_torque(quat, inertia_tensor, mean_motion):
    Ae3 = rotate_vec_with_quat(quat, np.array([0., 0., 1.]))

    return 3. * mean_motion ** 2. * cross_product(Ae3, inertia_tensor.dot(Ae3))    


def rhs(t, x, sat, action):

    quat = x[:4] / np.linalg.norm(x[:4])
    omega = x[4:7]
    h = x[7:10]
     
    omega_rel = omega - rotate_vec_with_quat(quat, np.array([0., sat.mean_motion, 0.]))    
        
    trq_gg = gravity_gradient_torque(quat, sat.J, sat.mean_motion)       

    x_dot = np.zeros(10)
    action = action.reshape(3,)
    x_dot[0] = -0.5 * quat[1:].dot(omega_rel)
    x_dot[1:4] = 0.5 * (quat[0] * omega_rel + cross_product(quat[1:], omega_rel))
    tmp3 = (trq_gg + action - cross_product(omega, sat.J.dot(omega)))
    x_dot[4:7] = sat.J_inv.dot(tmp3)
    x_dot[7:10] = - action

    return x_dot

In [ ]:
from types import SimpleNamespace
const = SimpleNamespace()
const.mu = 3.986e14
const.Re = 6037.1e3
sat = SimpleNamespace()
sat.J = np.diag(np.array([4, 5, 3]))
sat.J_inv = np.linalg.inv(sat.J)
sat.altitude = 600e3
sat.mean_motion = np.sqrt(const.mu / (const.Re + sat.altitude)**3.)


class TorqueDynamics(gym.Env):
  observation_space = gym.spaces.Box(-1, 1, shape=(10,))

  
  def __init__(self, dt, q_req, add_9=0.25):
    self.state = None
    self.dt = dt
    self.q_req = q_req
    self.q_req_conj = quat_conjugate(self.q_req)
    self.w_req = np.zeros(3)
    self.history = []
    self.t = []
    self.action_space = self.init_actions()
    self.q_prev = None
    self.add_9 = add_9

  @staticmethod
  def init_actions():
    a = np.linspace(-1, 1, 21)
    a1 = a/10
    a2 = a/100
    a = np.concatenate((a, a1, a2))
    a = np.unique(a.round(10))
    a = a[a>=-0.5]
    a = a[a<=0.5]

    s = a.shape
    a = np.vstack((a, np.zeros(s)))
    a = np.vstack((a, np.zeros(s)))
    a = a.T
    aroll1 = np.roll(a, 1)
    aroll2 = np.roll(a, 2)
    a = np.concatenate((a, aroll1))
    a = np.concatenate((a, aroll2))
    a = np.unique(a, axis=0)
    return a

  def reset(self, state=None):
    if state is not None:
      self.state = state
    else:
      self.state = self.observation_space.sample()
      phi = self.state[0] * np.pi 
      self.state[0] = np.cos(phi / 2)
      self.state[1:4] = normalize(self.state[1:4]) * np.sin(phi / 2)
      self.state[4:] = 0
    self.history = [self.state]
    self.t = [0]
    self.q_prev = self.state[:4]
    return self.state

  @staticmethod
  def r_a(phi, q_current, q_prev):
    return np.exp(2-phi) if q_current > q_prev else np.exp(2-phi)/2


  def r_t(self, reward, phi):
    return reward + 9 if phi <= self.add_9 else reward

  def step(self, action):
    t0 = 0
    tf = self.dt
    x_0 = self.state
    if isinstance(action, int) or isinstance(action, np.int64):
      action = self.action_space[action].reshape(3,1)
    
    sol = solve_ivp(lambda t, x: rhs(t, x, sat, action), (t0,tf), x_0) #, t_eval=t_eval)
    x = sol.y.T
    t = sol.t[1:]
    observation = x[-1]
    observations = normalize(observation)
    time_ = self.t[-1] + self.dt
    self.state = observation
    self.history.append(observation)
    self.t.append(time_)

    # calculating rewards:
    q_current = observation[:4]
    q_error = quat_product(self.q_req_conj, q_current)
    q_error = np.clip(q_error, -1, 1)
    w_current = observation[4:7]
    #print('q_error', q_error)
    phi = 2*np.arccos(q_error[0])
    r_inter = self.r_a(phi, q_current[0], self.q_prev[0])
    r1 = self.r_t(r_inter, phi)
    # r2 = -np.sum(np.abs(observation[4:7]))
    # Qreward = np.exp(-0.1 * np.linalg.norm(q_current - self.q_req))
    # Wreward = np.exp(-0.1 * np.linalg.norm(w_current - self.w_req))
    # reward = Qreward * Wreward
    # reward = self.r_t(reward, phi)
    reward = r_inter #r1 #+ 10 * r2
    #print('rewards', r1, r2)
    self.q_prev = q_current

    q_req_ext = np.concatenate([self.q_req, self.w_req])
    done = np.linalg.norm(observation[:7] - q_req_ext) < 1e-4
    
    info = dict()
    info['x'] = self.history
    info['t'] = self.t
    return observation, reward, done, info
  
  def render(self):
    pass

## Initializations

In [ ]:
# Initialize the environment and get the dimensionality of the
# observation space and the number of possible actions
env = TorqueDynamics(0.1, np.array([1, 0, 0, 0]), 0.05)
observation_dimensions = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]

# Initialize the buffer
buffer = Buffer(observation_dimensions, steps_per_epoch)

# Initialize the actor and the critic as keras models
observation_input = keras.Input(shape=(observation_dimensions,), dtype=tf.float32)
logits = mlp(observation_input, list(hidden_sizes) + [num_actions], tf.nn.relu, None)
actor = keras.Model(inputs=observation_input, outputs=logits)
value = tf.squeeze(
    mlp(observation_input, list(hidden_sizes) + [1], tf.nn.relu, None), axis=1
)
critic = keras.Model(inputs=observation_input, outputs=value)

# Initialize the policy and the value function optimizers
policy_optimizer = keras.optimizers.Adam(learning_rate=policy_learning_rate)
value_optimizer = keras.optimizers.Adam(learning_rate=value_function_learning_rate)



## Train

In [ ]:
def train(epochs, add_9, start=None):
# Iterate over the number of epochs
  print('starting training with', epochs, 'epochs')
  buffer = Buffer(observation_dimensions, steps_per_epoch)
  env = TorqueDynamics(0.1, np.array([1, 0, 0, 0]), add_9)
  # Initialize the observation, episode return and episode length
  observation, episode_return, episode_length = env.reset(start), 0, 0

  returns = []

  for epoch in range(epochs):
      # Initialize the sum of the returns, lengths and number of episodes for each epoch
      sum_return = 0
      sum_length = 0
      num_episodes = 0

      # Iterate over the steps of each epoch
      for t in range(steps_per_epoch):
          if render:
              env.render()

          # Get the logits, action, and take one step in the environment
          observation = observation.reshape(1, -1)
          logits, action = sample_action(observation)
          #print(type(action[0].numpy()))
          observation_new, reward, done, info = env.step(action[0].numpy())
          episode_return += reward
          episode_length += 1

          # Get the value and log-probability of the action
          value_t = critic(observation)
          logprobability_t = logprobabilities(logits, action)

          # Store obs, act, rew, v_t, logp_pi_t
          buffer.store(observation, action, reward, value_t, logprobability_t)

          # Update the observation
          observation = observation_new

          # Finish trajectory if reached to a terminal state
          terminal = done
          if terminal or (t == steps_per_epoch - 1):
              last_value = 0 if done else critic(observation.reshape(1, -1))
              buffer.finish_trajectory(last_value)
              sum_return += episode_return
              sum_length += episode_length
              num_episodes += 1
              observation, episode_return, episode_length = env.reset(start), 0, 0

      # Get values from the buffer
      (
          observation_buffer,
          action_buffer,
          advantage_buffer,
          return_buffer,
          logprobability_buffer,
      ) = buffer.get()

      # Update the policy and implement early stopping using KL divergence
      for _ in range(train_policy_iterations):
          kl = train_policy(
              observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
          )
          if kl > 1.5 * target_kl:
              # Early Stopping
              break

      # Update the value function
      for _ in range(train_value_iterations):
          train_value_function(observation_buffer, return_buffer)

      # Print mean return and length for each epoch
      print(
          f" Epoch: {epoch + 1}. Mean Return: {sum_return / num_episodes}. Mean Length: {sum_length / num_episodes}"
      )
      returns.append(sum_return / num_episodes)
  return returns

In [ ]:
print('Accounting for the gravity')
iterations = 20
reward_thresh = 0.1
tries = 100

for i in range(tries):
  print(f'Run #{i}. The "+9" threshold is {reward_thresh}')
  mean_ret = train(iterations, reward_thresh)
  mean_ret = np.min(mean_ret)
  print(f'Current batch mean is {mean_ret}')
  print('-'*10)
  if mean_ret > 5000 and reward_thresh > 0.004:
    reward_thresh *= 0.5# - 3*(mean_ret - 1000)/40000 # linear from (1000, 0.8) to (5000, 0.5)
  

In [ ]:
observation = env.reset()
episode_return = 0
# Iterate over the steps of each epoch
for t in range(steps_per_epoch*2):
    if render:
        env.render()

    # Get the logits, action, and take one step in the environment
    observation = observation.reshape(1, -1)
    logits, action = sample_action(observation)
    #print(type(action[0].numpy()))
    observation_new, reward, done, info = env.step(action[0].numpy())
    episode_return += reward

    # Update the observation
    observation = observation_new
print(episode_return)

In [ ]:
x = np.array(info['x'])
t = np.array(info['t'])
roll, pitch, yaw = quat2rpy_deg(x[:,0], x[:,1], x[:,2], x[:,3])
size = roll.shape[0]//3
print(np.mean(roll[-size:]))
print(np.mean(pitch[-size:]))
print(np.mean(yaw[-size:]))
print(np.std(roll[-size:]))
print(np.std(pitch[-size:]))
print(np.std(yaw[-size:]))

plot_euler_angles(t, roll, pitch, yaw, 'angles2.pdf')

In [ ]:
start = np.array([np.cos(0.001),0,0,np.sin(0.001),0,0,0,0,0,0])

In [ ]:
x = np.array(info['x'])
t = np.array(info['t'])
roll, pitch, yaw = quat2rpy(x[:,0], x[:,1], x[:,2], x[:,3])

plot_euler_angles_radial(t, roll, pitch, yaw, 'angles2.pdf')

In [ ]:
roll, pitch, yaw = quat2rpy_deg(x[:,0], x[:,1], x[:,2], x[:,3])
   
fig1 = plt.figure(figsize=(16,8))
ax1 = fig1.add_subplot(2,2,1)

ax1.set_title("Euler Angles")
ax1.plot(t, roll, label = 'roll', color = 'red')
ax1.plot(t, pitch, label = 'pitch', color = 'green')
ax1.plot(t, yaw, label = 'yaw', color = 'blue')
ax1.set_ylabel(r'angles, [deg]')
ax1.set_xlabel(r't, [s]')
ax1.grid(True)
ax1.legend()

ax2 = fig1.add_subplot(2,2,2)

ax2.set_title("Angular Velocity")
ax2.plot(t, x[:,4], label = '$\omega_x$', color = 'red')
ax2.plot(t, x[:,5], label = '$\omega_y$', color = 'green')
ax2.plot(t, x[:,6], label = '$\omega_z$', color = 'blue')
ax2.set_ylabel(r'angular velocity, [rad/s]')
ax2.set_xlabel(r't, [s]')
ax2.grid(True)
ax2.legend()

ax2 = fig1.add_subplot(2,2,3)

ax2.set_title("RW angular momentum")
ax2.plot(t, x[:,7], label = '$h_x$', color = 'red')
ax2.plot(t, x[:,8], label = '$h_y$', color = 'green')
ax2.plot(t, x[:,9], label = '$h_z$', color = 'blue')
ax2.set_ylabel(r'h, [Nms]')
ax2.set_xlabel(r't, [s]')
ax2.grid(True)
ax2.legend()

plt.tight_layout()

## Saving the weights

In [ ]:
actor_config = actor.get_config()
actor_weights = actor.get_weights()
critic_config = critic.get_config()
critic_weights = critic.get_weights()
import pickle
name = f'configs_weights{iterations} {reward_thresh}.pickle'
with open(name, 'wb') as f:
  pickle.dump((actor_config, actor_weights, critic_config, critic_weights), f)
from google.colab import files
files.download(name)

## Importing the weights

In [ ]:
import pickle
# iterations = 1500
# reward_thresh = 0.0043
name = f'configs_weights_gravity_exp.pickle'
with open(name, 'rb') as f:
  something = pickle.load(f)

actor_config, actor_weights ,critic_config, critic_weights = something
actor = keras.Model.from_config(actor_config)
actor.set_weights(actor_weights)
critic = keras.Model.from_config(critic_config)
critic.set_weights(critic_weights)